# Requirements 


In [ ]:
# before run this code, have to installed 
# Visual Studio C/C++
# cmake
# dlib
# face_recognition
# numpy
# opencv-python

# https://github.com/ageitgey/face_recognition

In [275]:
import face_recognition as fr
import os
import cv2
import numpy as np
from time import sleep
import matplotlib.pyplot as plt
from PIL import Image

In [299]:
def extractFace(src,dest):    
    image=fr.load_image_file(src)
    locations=fr.face_locations(image)

    for face_location in locations:
        top,right,bottom,left=face_location
        face=image[top:bottom,left:right]
        pil_image=Image.fromarray(face)
    #     image.show()
        pil_image.save(f"./{dest}/{top}.jpg")

src='Known/Jarif/c.jpg'
dest='Known/Jarif/'

extractFace(src,dest)

In [300]:

KNOWN_FACES_DIR = 'Known'
UNKNOWN_FACES_DIR = 'Unknown'


def getData():
    
    known_faces = []
    known_names = []

    # We oranize known faces as subfolders of KNOWN_FACES_DIR
    # Each subfolder's name becomes our label (name)
    for name in os.listdir(KNOWN_FACES_DIR):

        # Next we load every file of faces of known person
        for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
            
            if filename.endswith(".jpg") or filename.endswith(".png"):

                # Load an image
                image = fr.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')

                # Get 128-dimension face encoding
                # Always returns a list of found faces, for this purpose we take first face only (assuming one face per image as you can't be twice on one image)
                face_bouding_box=fr.face_encodings(image)
                if len(face_bouding_box)>0:                    
                    encoding = face_bouding_box[0]
                    # Append encodings and name
                    known_faces.append(encoding)
                    known_names.append(name)
                    print("File Name ",name," : ",filename)
                else:
                    print("No face detected : ",filename)
            
    return known_faces,known_names


In [301]:
faces_encoded,known_face_names=getData()
np.savez_compressed('known_faces.npz',faces_encoded,known_face_names)

File Name  Ahanaf  :  116.jpg
File Name  Anayet  :  70.jpg
File Name  Arif  :  102.jpg
File Name  Jarif  :  110.jpg
No face detected :  116.jpg
File Name  Jarif  :  139.jpg
File Name  Jarif  :  76.jpg
File Name  Jewel  :  1.jpg
File Name  Jewel  :  2.jpg
File Name  Mamta  :  1.jpg
File Name  Saran  :  66.jpg


In [302]:
dataset=np.load('known_faces.npz')
faces_encoded,known_face_names=dataset['arr_0'],dataset['arr_1']


In [323]:
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2

def classify_face(im):

    img = cv2.imread(im, 1)
    #img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
    #img = img[:,:,::-1]
 
    face_locations = fr.face_locations(img) # model='cnn', it will took long time to detect
    unknown_face_encodings = fr.face_encodings(img, face_locations)

    face_names = []
    for face_encoding in unknown_face_encodings:
        # See if the face is a match for the known face(s)
        matches = fr.compare_faces(faces_encoded, face_encoding,TOLERANCE) 
        name = "Unknown"

        # use the known face with the smallest distance to the new face
        face_distances = fr.face_distance(faces_encoded, face_encoding)        
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            

        face_names.append(name)

        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Draw a box around the face
            cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 0), 2)

            # Draw a label with a name below the face
            cv2.rectangle(img, (left, bottom+12 ), (right, bottom), (0, 0, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(img, name, (left, bottom+10), font, 0.5, (255, 255, 255), 1)


            
    image=Image.fromarray(img)
    image.show()  
    return face_names 









In [324]:
print(classify_face("Unknown/1.jpg"))

['Jarif']
